<a href="https://colab.research.google.com/github/AryaPratamaPutra-10/Analyst-Uang/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

HSI

fungsi otsu_thresholding = untuk melakukan segmentasi gambar menggunakan metode otsu's thresholding. Sederhananya, metode ini akan mengubah gambar berwarna atau grayscale menjadi gambar hitam putih (binary) dengan cara menentukan nilai ambang batas (threshold) yang optimal.

In [ ]:
def otsu_thresholding(image_path, output_size=(300, 300)):
    # Baca gambar
    original_image = cv2.imread(image_path)

    # Konversi gambar ke model warna HSI
    hsi_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)

    # Ambil saluran intensitas (komponen I dari HSI)
    intensity_channel = hsi_image[:, :, 2]

    # Resize gambar asli
    resized_original = cv2.resize(intensity_channel, output_size)

    # Hitung histogram
    hist = cv2.calcHist([intensity_channel], [0], None, [256], [0, 256])

    # Normalisasi histogram
    hist_norm = hist.ravel() / hist.sum()

    # Hitung kumulatif histogram
    Q = hist_norm.cumsum()

    # Hitung kumulatif mean
    bins = np.arange(256)
    mean = np.sum(bins * hist_norm)

    # Hitung variance antara kelas
    variance = np.sum((bins - mean) ** 2 * hist_norm)

     # Inisialisasi variabel
    threshold = -1
    max_sigma = -1

    # Iterasi melalui nilai threshold yang mungkin
    for i in range(1, 256):
        p1, p2 = np.hsplit(hist_norm, [i])  # Pisahkan dua kelas
        q1 = Q[i]
        q2 = Q[255] - Q[i]

        # Hindari pembagian nol
        if q1 == 0 or q2 == 0:
            continue

        # Hitung variance intra kelas
        b1 = np.sum(p1 * bins[:i]) / q1
        b2 = np.sum(p2 * bins[i:]) / q2
        sigma = q1 * q2 * ((b1 - b2) ** 2)

        # Update threshold jika ditemukan sigma yang lebih besar
        if sigma > max_sigma:
            max_sigma = sigma
            threshold = i

    # Thresholding gambar menggunakan nilai threshold yang dihitung
    _, thresholded = cv2.threshold(intensity_channel, threshold, 255, cv2.THRESH_BINARY)

    # Resize gambar hasil thresholding
    resized_thresholded = cv2.resize(thresholded, output_size)
    return resized_original, resized_thresholded

def detect_currency(image_path):
    # Baca gambar
    image = cv2.imread(image_path)

    # Inisialisasi detektor objek
    detector = cv2.SimpleBlobDetector_create()

    # Konversi gambar ke skala abu-abu
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Deteksi blob (objek)
    keypoints = detector.detect(gray)

    # Menentukan apakah objek terdeteksi sebagai uang asli atau palsu
    if len(keypoints) > 0:
        result = "Uang Asli"
    else:
        result = "Uang Palsu"

    # Mengembalikan hasil deteksi
    return result